In [9]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [8]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [4]:
df = gpd.read_file('data/attributesWithLabels_q02.json')

In [5]:
df

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,image_date,label_date,label_severity,label_is_temporary,agree_count,disagree_count,notsure_count,label_description,user_id,geometry
0,67982054,SurfaceProblem,6316,6449725,Laurelhurst,3.0,False,165367,lyPVrWTOVxRWc37_HquiXA,137.718750,...,2019-06,2021-07-14 13:41:40.121,4.0,False,2,0,0,None,1878bcff-d89c-4b3d-8162-0eba4bb70b45,POINT (-122.27452 47.65587)
1,68013712,CurbRamp,10890,6521940,Green Lake,2.0,False,141953,ybLKz3WqjD3CP9GRyZCFeg,216.218750,...,2019-05,2021-03-23 22:58:29.465,1.0,False,1,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32298 47.68078)
2,68015448,NoCurbRamp,5896,6442919,Green Lake,4.0,False,141936,sgnFUUgo5BsnUUG1N68ArA,124.671875,...,2019-05,2021-03-23 22:52:54.957,4.0,False,2,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32279 47.68198)
3,68011383,CurbRamp,21088,336908262,Madrona,3.0,False,143129,tG1goNgIa9fSKBuf57ecWg,169.000000,...,2019-06,2021-03-31 22:39:26.170,3.0,False,1,0,0,None,f0cf6e59-ea98-417a-b02c-c7f18f130de3,POINT (-122.29475 47.61882)
4,67930050,NoCurbRamp,1022,6349133,Harrison/Denny-Blaine,3.0,False,143135,tG1goNgIa9fSKBuf57ecWg,42.062500,...,2019-06,2021-03-31 22:40:37.353,3.0,False,2,0,0,None,f0cf6e59-ea98-417a-b02c-c7f18f130de3,POINT (-122.29478 47.61897)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38594,67996004,CurbRamp,17520,171739214,Broadway,2.0,False,105565,xGSGC2QVfhSYS0ho9IF7HA,21.500000,...,2019-05,2020-06-23 07:48:00.924,1.0,False,1,0,0,None,af812204-1521-4c42-bf88-4baaaffe3f06,POINT (-122.32206 47.62114)
38595,67998083,NoCurbRamp,1983,6368518,Broadway,4.0,False,105653,f9VxkXoPw0wYyJ4BzAZM9g,122.500000,...,2019-05,2020-06-23 08:10:18.895,5.0,False,1,1,0,None,af812204-1521-4c42-bf88-4baaaffe3f06,POINT (-122.32523 47.62506)
38596,67998102,NoCurbRamp,939,6348400,Broadway,5.0,False,105681,rhQMq1gFHtCp3XQhFEtjtw,278.500000,...,2019-05,2020-06-23 08:18:00.588,5.0,False,2,0,0,None,af812204-1521-4c42-bf88-4baaaffe3f06,POINT (-122.32432 47.62769)
38597,67997698,Obstacle,5050,6428544,Broadway,3.0,False,105712,TrBuXhClTi0f1q_8ykqCTg,60.250000,...,2019-05,2020-06-23 08:22:30.016,3.0,False,1,1,0,None,af812204-1521-4c42-bf88-4baaaffe3f06,POINT (-122.32298 47.62877)


In [10]:
# break geometry into lat and lng
df['lat'] = df.geometry.y
df['lng'] = df.geometry.x

In [11]:
df0 = cluster_label_type_at_index(df,0)[2]

In [12]:
df1 = cluster_label_type_at_index(df,1)[2]

In [13]:
df2 = cluster_label_type_at_index(df,2)[2]

In [14]:
df3 = cluster_label_type_at_index(df,3)[2]

In [15]:
df4 = cluster_label_type_at_index(df,4)[2]

In [16]:
df5 = cluster_label_type_at_index(df,5)[2]

In [18]:
df6 = cluster_label_type_at_index(df,6)[2]

In [19]:
df7 = cluster_label_type_at_index(df,7)[2]

In [20]:
df8 = cluster_label_type_at_index(df,8)[2]

In [21]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [22]:
df0

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,agree_count,disagree_count,notsure_count,label_description,user_id,geometry,lat,lng,cluster_id,clustered
1,68013712,CurbRamp,10890,6521940,Green Lake,2.0,False,141953,ybLKz3WqjD3CP9GRyZCFeg,216.218750,...,1,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32298 47.68078),47.680782,-122.322975,6649,True
3,68011383,CurbRamp,21088,336908262,Madrona,3.0,False,143129,tG1goNgIa9fSKBuf57ecWg,169.000000,...,1,0,0,None,f0cf6e59-ea98-417a-b02c-c7f18f130de3,POINT (-122.29475 47.61882),47.618820,-122.294754,4873,False
6,68013828,CurbRamp,4553,6418745,Green Lake,3.0,False,143292,e3kjoVGSoXdOWJAmziiluw,295.140625,...,1,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32533 47.68117),47.681171,-122.325325,2139,True
8,67735347,CurbRamp,4163,6411917,Montlake,1.0,False,143563,zsNZjN4yeqDuUMTHOWc3dQ,246.000000,...,1,0,0,None,12d3f3e7-1635-41ee-84e5-28f4130828d5,POINT (-122.31734 47.63820),47.638199,-122.317337,8772,True
9,67735502,CurbRamp,5392,6434062,Montlake,2.0,False,143577,hndJUVVwgBako7m3BiALgQ,171.562500,...,1,0,0,None,12d3f3e7-1635-41ee-84e5-28f4130828d5,POINT (-122.29954 47.64309),47.643085,-122.299538,7638,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38587,67995440,CurbRamp,391,6343949,Broadway,1.0,False,110646,FvV5HPbWo--iWSVG7XN1zQ,211.267853,...,1,0,0,None,ec15a589-dd14-4513-a43e-8c06e55f4c71,POINT (-122.31702 47.61641),47.616405,-122.317017,33,True
38590,68011137,CurbRamp,18795,243349945,Madrona,1.0,False,88178,BIj6ffRSTokeEKMvEWgcMQ,235.026779,...,1,0,0,None,09474566-a695-4259-85ed-7d24f3d7a70d,POINT (-122.28819 47.61646),47.616455,-122.288193,23,False
38591,68022235,CurbRamp,13692,38502228,University District,2.0,False,95626,AJ7hqnoaPy5f2gWuPdAvQg,302.750000,...,1,0,0,None,81ef8000-218e-4a4d-bd4c-fc2eac1990f3,POINT (-122.30557 47.66314),47.663139,-122.305573,12,True
38594,67996004,CurbRamp,17520,171739214,Broadway,2.0,False,105565,xGSGC2QVfhSYS0ho9IF7HA,21.500000,...,1,0,0,None,af812204-1521-4c42-bf88-4baaaffe3f06,POINT (-122.32206 47.62114),47.621140,-122.322060,8,True


In [23]:
#concatenate all dataframes
clusters_q2 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [24]:
clusters_q2

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,agree_count,disagree_count,notsure_count,label_description,user_id,geometry,lat,lng,cluster_id,clustered
1,68013712,CurbRamp,10890,6521940,Green Lake,2.0,False,141953,ybLKz3WqjD3CP9GRyZCFeg,216.218750,...,1,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32298 47.68078),47.680782,-122.322975,6649,True
3,68011383,CurbRamp,21088,336908262,Madrona,3.0,False,143129,tG1goNgIa9fSKBuf57ecWg,169.000000,...,1,0,0,None,f0cf6e59-ea98-417a-b02c-c7f18f130de3,POINT (-122.29475 47.61882),47.618820,-122.294754,4873,False
6,68013828,CurbRamp,4553,6418745,Green Lake,3.0,False,143292,e3kjoVGSoXdOWJAmziiluw,295.140625,...,1,0,0,None,8ba4641d-f661-4351-8f20-ff86057a8b16,POINT (-122.32533 47.68117),47.681171,-122.325325,2139,True
8,67735347,CurbRamp,4163,6411917,Montlake,1.0,False,143563,zsNZjN4yeqDuUMTHOWc3dQ,246.000000,...,1,0,0,None,12d3f3e7-1635-41ee-84e5-28f4130828d5,POINT (-122.31734 47.63820),47.638199,-122.317337,8772,True
9,67735502,CurbRamp,5392,6434062,Montlake,2.0,False,143577,hndJUVVwgBako7m3BiALgQ,171.562500,...,1,0,0,None,12d3f3e7-1635-41ee-84e5-28f4130828d5,POINT (-122.29954 47.64309),47.643085,-122.299538,7638,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23442,67952672,Signal,3987,6407638,Maple Leaf,NaN,False,207739,lDWrC9a4jVuYeZXIsPiuUw,301.625000,...,7,0,3,None,d57a2ee7-6b2c-4ef4-a04d-c8d688f78ca2,POINT (-122.32318 47.69595),47.695950,-122.323181,6,False
29258,67952674,Signal,16307,158719608,Maple Leaf,NaN,False,231609,jVLp7QRXH9TyDjbUXpXTTw,358.750000,...,0,0,0,None,c11b44ac-3fa8-45f8-acf0-de27da3e1428,POINT (-122.32037 47.69416),47.694164,-122.320374,1,False
32983,67952683,Signal,23577,428088700,Maple Leaf,NaN,False,211499,X2vwDniHFME9i76PRc8PeA,218.250000,...,4,0,1,None,4cac48cc-452d-40c1-8f16-9f54d6da1a6d,POINT (-122.31804 47.70842),47.708420,-122.318039,15,False
35544,67952673,Signal,21671,362331765,Maple Leaf,NaN,False,211434,PXwgPk0LVKmuuUFmiC5OhA,263.625000,...,4,1,0,None,4cac48cc-452d-40c1-8f16-9f54d6da1a6d,POINT (-122.32598 47.70306),47.703056,-122.325981,4,False


In [25]:
#export to geojson
clusters_q2.to_file("data/clusters_q2.geojson", driver='GeoJSON')